In [1]:
!pip install -U transformers

/bin/bash: pip: command not found


In [54]:
from tokenizers import BertWordPieceTokenizer

wb_tokenizer = BertWordPieceTokenizer(clean_text=True, handle_chinese_chars=True,
                                      strip_accents=True, lowercase=True)

wb_tokenizer.train('./data/ka_nse_mil.txt',
                   vocab_size=20000, min_frequency=5,
                   special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
# wb_tokenizer.save_model(".")

In [55]:
wb_tokenizer.encode('მაგარია ძააააააან. ძვ.წ.').tokens

['მაგარია', 'ძა', '##აააა', '##აა', '##ნ', '.', 'ძვ', '.', 'წ', '.']

In [56]:
import nltk
from typing import Iterator, List
from nltk import RegexpTokenizer, sent_tokenize
from tqdm import tqdm
from pathlib import Path

nltk.download('punkt')

import re
_patterns = [r"<br \/>", r",", r"\(", r"\)", r"\„", r"\“"]

_replacements = [" ", " , ", " ( ", " ) ", " \" ", " \" "]

_patterns_dict = list((re.compile(p), r) for p, r in zip(_patterns, _replacements))


def generate_tokenized_sentences(path: Path, paragraphs=1_000_000) -> Iterator[str]:
    """
    Tokenize paragraphs into sentences, and sentences into words.
    :param paragraph: text of paragraph
    """
    word_tokenizer = RegexpTokenizer(r'\b\d+[/\.]\d+[/\.]\d+|ძვ\.წ\.|ახ\.წ\.|ე\.წ\.|ა\.შ\.|ე\.ი\.|[-\w]+|[-!\".#$%&\'()*+,/:;<=>?@\[\\\]^_`{|}~„“+]+')

    # for sentence in sent_tokenize(paragraph):
    with open(path, 'r') as file:
        for i, paragraph in enumerate(file):
            if i >= paragraphs:
                break
                
            paragraph = paragraph.lower()
            for pattern_re, replaced_str in _patterns_dict:
                paragraph = pattern_re.sub(replaced_str, paragraph)

            tokenized_sentence = word_tokenizer.tokenize(paragraph)
            if tokenized_sentence:
                start = 0
                for i, tkn in enumerate(tokenized_sentence):
                    if tkn in ('.', '!', '?'):
                        tmp = []
                        for word in tokenized_sentence[start: i+1]:
                            tmp += wb_tokenizer.encode(word).tokens
                        yield tmp
                        start = i+1

                if start < len(tokenized_sentence):
                    tmp = []
                    for word in tokenized_sentence[start:]:
                        tmp += wb_tokenizer.encode(word).tokens
                    yield tmp

[nltk_data] Downloading package punkt to /home/khokho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [69]:
from pathlib import Path

temp_data_path = Path('data/ka_nse_mil.txt')

tokenized_sentences = [sent for sent in generate_tokenized_sentences(temp_data_path)]
tokenized_sentences[0]

['ძეგლთა',
 'დაცვის',
 'ეროვნულ',
 'სააგენტო',
 '##ში',
 'აცხადებენ',
 ',',
 'რომ',
 'მათ',
 'მიერ',
 '"',
 'სამხრეთის',
 'კარიბჭე',
 '##სთ',
 '##ვის',
 '"',
 'მოწოდ',
 '##ებული',
 'ინფორმაცია',
 ',',
 'იმის',
 'შესახებ',
 'რომ',
 'ახალციხის',
 'არქი',
 '##ვის',
 'ეზოში',
 'აღმოჩენილი',
 'შენობა',
 'საკ',
 '##ულტ',
 '##ო',
 'ნაგებობა',
 'იყო',
 'არასწორი',
 '##ა',
 'და',
 'საბოლოოდ',
 'დადგინდა',
 ',',
 'რომ',
 'აქ',
 'ტრადიციული',
 'მესხ',
 '##ური',
 'საცხოვრებელი',
 'იყო',
 '.']

In [70]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline

# adds padding to sentences, on the left and on the right
# train contains ngrams (if n=3, train contains 1-grams, 2-grams and 3-grams)
# vocab contains padded and flattened sentences
temp_data_path = Path('data/ka_nse_mil.txt')
n = 6
train, vocab = padded_everygram_pipeline(n, tokenized_sentences)

In [71]:
from nltk.lm import MLE, Laplace
# model = MLE(n)
# TODO: compare different lm models (laplace seems better than mle)
model = Laplace(n)

In [72]:
model.fit(train, vocab)
print(model.vocab)
print(model.counts)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 19388 items>
<NgramCounter with 6 ngram orders and 84015153 ngrams>


In [117]:
tst_sent = next(generate_tokenized_sentences(temp_data_path))
print(model.vocab.lookup(tst_sent))
print(model.vocab.lookup(tst_sent + ['salfknoonqwf'])) # check unknown tokens

('ძეგლთა', 'დაცვის', 'ეროვნულ', 'სააგენტო', '##ში', 'აცხადებენ', ',', 'რომ', 'მათ', 'მიერ', '"', 'სამხრეთის', 'კარიბჭე', '##სთ', '##ვის', '"', 'მოწოდ', '##ებული', 'ინფორმაცია', ',', 'იმის', 'შესახებ', 'რომ', 'ახალციხის', 'არქი', '##ვის', 'ეზოში', 'აღმოჩენილი', 'შენობა', 'საკ', '##ულტ', '##ო', 'ნაგებობა', 'იყო', 'არასწორი', '##ა', 'და', 'საბოლოოდ', 'დადგინდა', ',', 'რომ', 'აქ', 'ტრადიციული', 'მესხ', '##ური', 'საცხოვრებელი', 'იყო', '.')
('ძეგლთა', 'დაცვის', 'ეროვნულ', 'სააგენტო', '##ში', 'აცხადებენ', ',', 'რომ', 'მათ', 'მიერ', '"', 'სამხრეთის', 'კარიბჭე', '##სთ', '##ვის', '"', 'მოწოდ', '##ებული', 'ინფორმაცია', ',', 'იმის', 'შესახებ', 'რომ', 'ახალციხის', 'არქი', '##ვის', 'ეზოში', 'აღმოჩენილი', 'შენობა', 'საკ', '##ულტ', '##ო', 'ნაგებობა', 'იყო', 'არასწორი', '##ა', 'და', 'საბოლოოდ', 'დადგინდა', ',', 'რომ', 'აქ', 'ტრადიციული', 'მესხ', '##ური', 'საცხოვრებელი', 'იყო', '.', '<UNK>')


In [74]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

# detokenize = TreebankWordDetokenizer().detokenize

def generate_sentence(model, num_words=20, text_seed=None, random_seed=None):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max num of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    text_seed_tokenized = []
    if text_seed:
        for word in text_seed:
            text_seed_tokenized += wb_tokenizer.encode(word).tokens
    
    # print(text_seed_tokenized)
    for token in model.generate(num_words, text_seed=text_seed_tokenized, random_seed=random_seed):
        # print(token)
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
        
    # print(content)
    decoded = wb_tokenizer.decode([wb_tokenizer.token_to_id(tok) for tok in text_seed_tokenized + content])

    return decoded

In [91]:
generate_sentence(model, num_words=20, text_seed=['ეს', 'არის'], random_seed=78)

'ეს არის ფინალისტების ყველაზე მოკლე სია ბოლო წლების განმავლობაში.'

In [96]:
generate_sentence(model, num_words=100, text_seed=['ჩემი', 'ხატია', 'სამშობლო'], random_seed=78)

'ჩემი ხატია სამშობლოების, იმპერიალისტური სახელმწიფოების, და უწინარეს ყოვლისა დასავლეთისგან.'

In [112]:
# Pickled model generates the same sentence when using the same parameters.
generate_sentence(model, num_words=20, text_seed=['ქართველი', 'ერი', 'ამას', 'არ', 'აიტანს'], random_seed=78)

'ქართველი ერი ამას არ აიტანს მას სართულებზე.'

In [104]:
generate_sentence(model, num_words=100, text_seed=['გიორგი', 'სახლში'], random_seed=78)

'გიორგი სახლში სულ პოლზე ფიქრობდა დაჟინებით, იქნებ, ჩემმა ფიქრებმა პარიზამდე მიაღწიოსო!'

In [106]:
generate_sentence(model, num_words=100, text_seed=['რა', 'გინდა'], random_seed=78)

'რა გინდა მოვხატოო?'

In [115]:
for i in range(5):
    print(generate_sentence(model, num_words=100, text_seed=['გიორგი', 'სახლში']))

გიორგი სახლში გარდაიცვალა.
გიორგი სახლში.
გიორგი სახლში, ყავის ჭიქით ხელში, კიდევ ერთი გემრიელად სასუსნავი წიგნი შემოგვთავაზა, რომელიც ვებ - სერფინგის არსებითად გაუმჯობესებას ემსახურება.
გიორგი სახლში ვარ, არ ვმუშაობ ვნებივრობ!
გიორგი სახლში ძაღლი მიიყვანა, უბედური შემთხვევის გამო შვილი გარდაიცვალა, დაღუპვამდე ეს ძაღლი დაკარგა, მაგრამ დაკრძალვის დღეს მივიდა ძაღლი და, ვინც იმ ბიჭს არ უყვარდა, ის ადამიანები სახლში არ შეუშვა.


In [82]:
import pickle 

with open('models/n-gram/ngram_subwords.pkl', 'wb') as fout:
    pickle.dump(model, fout)

In [118]:
test_data_path = Path('data/ka_nse_test.txt')
test_tokenized_sentences = [sent for sent in generate_tokenized_sentences(test_data_path)]
test_tokenized_sentences[0]

['191',
 '##2',
 'წლის',
 'მარტში',
 'ნე',
 '##აპ',
 '##ოლის',
 'ნავს',
 '##ადგურ',
 '##ში',
 'დიდი',
 'სა',
 '##ოკ',
 '##ეა',
 '##ნო',
 'გემი',
 '##ს',
 'გადმო',
 '##ტვირთ',
 '##ვისას',
 'მოხდა',
 'ერთი',
 'უცნაური',
 'უბედური',
 'შემთხვევა',
 ',',
 'რომლის',
 'თაობაზე',
 'გაზეთ',
 '##ებმა',
 'დაწვრილებით',
 ',',
 'მაგრამ',
 'ძალზე',
 'შელ',
 '##ამაზ',
 '##ებული',
 'ცნობები',
 'გამოაქვეყ',
 '##ნეს',
 '.']

In [122]:
from tqdm import tqdm

test_data, _ = padded_everygram_pipeline(n, test_tokenized_sentences)
perplexities = []

for test in tqdm(test_data):
    perplexities.append(model.perplexity(test))

128050it [03:09, 676.97it/s] 


[3353.548204650126,
 3927.6283543178456,
 1355.792879430093,
 2867.8217021790615,
 5216.8922074748025,
 2215.6726618459857,
 1397.2026027262311,
 3033.1615613438134,
 2266.462327582449,
 3988.8562228063047,
 2569.301081912651,
 1989.5143882977884,
 4734.126244755172,
 3661.212697400539,
 4004.119594745804,
 912.8661530610918,
 2235.452127401238,
 1334.6304783950925,
 1111.1837981511696,
 1498.7013106698096,
 3372.7269211707903,
 581.9148020381758,
 814.8632909072021,
 2292.4231609381486,
 3449.5482966617237,
 2582.759771704082,
 1323.4911764335125,
 1380.3269661458064,
 1145.1466283058073,
 1801.5402247546963,
 1303.9128501178338,
 2815.346636562194,
 972.6534854252866,
 3954.0952394369024,
 142.0103743440005,
 4752.8706149552645,
 5448.840987269923,
 4506.5375159286705,
 2973.886411302783,
 5380.421259214135,
 319.4765755786964,
 866.9339614305338,
 2889.17438592369,
 3662.991096600073,
 4054.184911876334,
 2530.273205433893,
 3321.960465678954,
 5503.293787629275,
 5202.570876643305,

In [142]:
import numpy as np

print(f'Average perplexity: {np.mean(perplexities)}\n')

ids_sorted = np.argsort(perplexities)
print(f'Non-trivial sentences with lowest perplexity:')
p = 0
for i in range(99999999):
    ii = ids_sorted[i]
    if len(test_tokenized_sentences[ii]) > 10:
        p += 1
        print(f'{wb_tokenizer.decode([wb_tokenizer.token_to_id(tok) for tok in test_tokenized_sentences[ii]])}\n\tScore: {perplexities[ii]}')
    if p >= 5:
        break

print(f'\nNon-trivial sentences with highest perplexity:')
p = 0
for i in range(99999999):
    ii = ids_sorted[-i]
    if len(test_tokenized_sentences[ii]) > 10:
        p += 1
        print(f'{wb_tokenizer.decode([wb_tokenizer.token_to_id(tok) for tok in test_tokenized_sentences[ii]])}\n\tScore: {perplexities[ii]}')
    if p >= 5:
        break

Average perplexity: 1062.9767968225563

Non-trivial sentences with lowest perplexity:
........................................................................................
	Score: 13.277543886751673
რა თქმა უნდა, ეს არ იყო ჩვეულებრივი აბები.
	Score: 104.56874232520806
- კი, რა თქმა უნდა, გელოდებათ.
	Score: 117.53215700741329
- არა, სერ, რა თქმა უნდა, არა!
	Score: 126.57067235906356
- არა, არა, არა, არა, არა!
	Score: 133.13735775393485

Non-trivial sentences with highest perplexity:
aber statt dessen, wie nutzlos muht er sich ab ; immer noch zwangt er sich durch die gemacher des innersten palastes ; niemals wird er sie uberwinden ; und gelange ihm dies, nichts ware gewonnen ; die treppen hinab mußte er sich kampfen ; und gelange ihm dies, nichts ware gewonnen ; die hofe waren zu durchmessen ; und nach den hofen der zweite umschließende palast ; und wieder treppen und hofe ; und wieder ein palast ; und so weiter durch jahrtausende ; und sturzte er endlich aus dem außersten tor - aber 